# [PyBroMo](http://tritemio.github.io/PyBroMo/) - Reference - Data format and internals

<small>
*This notebook is part of [PyBroMo](http://tritemio.github.io/PyBroMo/) a 
python-based single-molecule Brownian motion diffusion simulator 
for confocal [smFRET](http://en.wikipedia.org/wiki/Single-molecule_FRET)
experiments. You can find the full list of notebooks [here](http://nbviewer.ipython.org/github/tritemio/PyBroMo/tree/master/notebooks/).*
</small>

> **Summary.** *This notebook describes the HDF5 file format used to store simulation results. 
> It is also an example on how to use the pytables API to navigate through the HDF5 file.*

In [1]:
%run load_pybromo.py

C:\Data\Antonio\software\src\pybromo

PyBroMo revision:
 2014-07-25 af5a325 Move test methods to a new file



In [2]:
# Initialize the random state
rs = np.random.RandomState(1)
print 'Initial random state:', hash_(rs.get_state())

# Diffusion coefficient
Du = 12.0           # um^2 / s
D = Du*(1e-6)**2    # m^2 / s

# Simulation time step (seconds)
t_step = 0.5e-6

# Time duration of the simulation (seconds)
t_max = 0.3

# Simulation box definition
box = Box(x1=-4.e-6, x2=4.e-6, y1=-4.e-6, y2=4.e-6, z1=-6e-6, z2=6e-6)

# PSF definition
psf = NumericPSF()

# Particles definition
P = gen_particles(15, box, rs=rs)

# Particle simulation definition
S = ParticlesSimulation(D=D, t_step=t_step, t_max=t_max, particles=P, box=box, psf=psf)

print 'Current random state:', hash_(rs.get_state())
S

Initial random state: bfb867eb5c5858e04685b790d6370c458b9747d6
Current random state: a347679b154f41d3d6847933720004f51811c0e6


Box: X 8.0um, Y 8.0um, Z 12.0um
D 1.2e-11, #Particles 15, 32.4 pM, t_step 0.5us, t_max 0.3s ID_EID 0 0

#HDF5 File format

The simulation is saved in a HDF5 file, one for each running engine. The file has the following content:

- <big>**/parameters**</big>
    
    * Numeric parameters (storead as scalar arrays)
        - `D`
        - `t_step`
        - `t_max`
        - `EID`
        - `ID`
        - `chunksize`: used for `emission` and `position` arrays
        - `np`: number of particles
        - `pMol`: particles concentration in pico-Molar

    * Non-numeric parameters (stored as group attributes)
        - `box`: the `Box()` object (stores boundaries and size)
        - `particles`: the `Particles()` object, a list of `Particle()` 
           (containing the initial position  positions) and seed.


- <big>**/psf**</big>
    * Arrays of PSF used in the simulation. This is the raw array as saved from PSFLab. The name of the array is the same as the origina file name. The PSF array has the following attributes:
        - 'fname', 'dir_', 'x_step', 'z_step'
            * The array and its attributes allow to recreate the `NumericPSF()` object on a simulation reload.
        - **TODO**: 'wavelength', 'NA', and other PSFLab parameters
    * `default_psf`: hard link to the PSF used in the simualation, used as persistent name


- <big>**/trajectories**</big>
    * `emission`: 2D array of emission traces: one row per particle. Shape: (num_particles, time)
    * `emission_tot`: 1D array of emission trace: total emission from all the particles: Shape: (time)
    * `position`: 3D array of positions. Shape (num_particles, 3, time)


- <big>**/timestamps**</big>
    * Arrays of timestamps for different `rate_max`, `bg_rate` and `seed`.

## How to access

The HDF5 file handle is in `S.store.data_file` after you run `S.open_store()`:

In [3]:
S.open_store(chunksize=2**20)

In [4]:
# Simulate 3D diffusion and emission
S.sim_brownian_motion(total_emission=False, save_pos=True)

[PID 1344] Simulation chunk: . . . . . . . . .


In [5]:
# Generate timestamps
S.sim_timestamps_em_store(max_rate=3e5, bg_rate=2e3, seed=1)

INFO: Random state initialized from seed (1).


In [6]:
S.compact_name_core()

'7798be_D1.2e-11_15P_32pM_step0.5us'

In [7]:
S.compact_name_core(t_max=True)

'7798be_D1.2e-11_15P_32pM_step0.5us_t_max0.3s'

In [8]:
S.compact_name()

'7798be_D1.2e-11_15P_32pM_step0.5us_t_max0.3s_ID0-0'

#HDF5 File inspection

In [9]:
print 'Main groups:\n'
for node in S.store.data_file.root:
    print node
    for n in node:
        print '\t%s' % n.name
        print '\t    %s' % n.title

Main groups:

/parameters (Group) 'Simulation parameters'
	D
	    Diffusion coefficient (m^2/s)
	EID
	    IPython Engine ID (int)
	ID
	    Simulation ID (int)
	chunksize
	    Chunksize for arrays
	np
	    Number of simulated particles
	pico_mol
	    Particles concentration (pM)
	t_max
	    Simulation total time (s)
	t_step
	    Simulation time-step (s)
/psf (Group) 'PSFs used in the simulation'
	default_psf
	    PSF x-z slice (PSFLab array)
	xz_realistic_z50_150_160_580nm_n1335_HR2
	    PSF x-z slice (PSFLab array)
/timestamps (Group) 'Timestamps of emitted photons'
	max_rate300kcps_bg2000cps_rs_bfb8
	    Simulated photon timestamps
	max_rate300kcps_bg2000cps_rs_bfb8_par
	    Particle number for each timestamp
/trajectories (Group) 'Simulated trajectories'
	emission
	    Emission trace of each particle
	emission_tot
	    Summed emission trace of all the particles
	position
	    3-D position trace of each particle


In [10]:
dfile = S.store.data_file

In [11]:
dfile.root._v_attrs._f_list('all')

['CLASS', 'PYTABLES_FORMAT_VERSION', 'TITLE', 'VERSION']

In [12]:
dfile.root._v_attrs._v_attrnames

['CLASS', 'PYTABLES_FORMAT_VERSION', 'TITLE', 'VERSION']

In [13]:
'TITLE' in dfile.root._v_attrs

True

## Group: /parameters

In [14]:
group = '/parameters'

print 'Numeric attributes (nodes) in %s:\n' % group

print S.store.data_file.get_node(group)
for node in S.store.data_file.get_node(group)._f_list_nodes():
    print '\t%s (%s)' % (node.name, str(node.read()))
    print '\t    %s' % node.title

Numeric attributes (nodes) in /parameters:

/parameters (Group) 'Simulation parameters'
	D (1.2e-11)
	    Diffusion coefficient (m^2/s)
	EID (0)
	    IPython Engine ID (int)
	ID (0)
	    Simulation ID (int)
	chunksize (1048576)
	    Chunksize for arrays
	np (15)
	    Number of simulated particles
	pico_mol (32.4324023632)
	    Particles concentration (pM)
	t_max (0.3)
	    Simulation total time (s)
	t_step (5e-07)
	    Simulation time-step (s)


In [15]:
group = '/parameters'

print 'Attributes in %s:\n' % group

print S.store.data_file.get_node(group)
for attr in S.store.data_file.get_node(group)._v_attrs._f_list():
    print '\t%s' % attr
    print '\t    %s' % type(S.store.data_file.get_node(group)._v_attrs[attr])

Attributes in /parameters:

/parameters (Group) 'Simulation parameters'
	box
	    <type 'instance'>
	particles
	    <class '__main__.Particles'>


In [16]:
particles = S.store.data_file.root.parameters._f_getattr('particles')

In [17]:
len(particles), particles.rs_hash

(15, 'bfb')

## Group /psf

In [18]:
group = '/psf'

print 'Nodes in in %s:\n' % group

print S.store.data_file.get_node(group)
for node in S.store.data_file.get_node(group)._f_list_nodes():
    print '\t%s %s' % (node.name, node.shape)
    print '\t    %s' % node.title

Nodes in in /psf:

/psf (Group) 'PSFs used in the simulation'
	default_psf (193, 129)
	    PSF x-z slice (PSFLab array)
	xz_realistic_z50_150_160_580nm_n1335_HR2 (193L, 129L)
	    PSF x-z slice (PSFLab array)


###`default_psf` attributes

In [19]:
node_name = '/psf/default_psf'
node = S.store.data_file.get_node(node_name)

print "\n%s %s: '%s'" % (node.name, node.shape, node.title)
print '\n    List of attributes:'
for attr in node.attrs._f_list():
    print '\t%s' % attr
    print "\t    %s" % repr(node._v_attrs[attr])


default_psf (193, 129): 'PSF x-z slice (PSFLab array)'

    List of attributes:
	dir_
	    'psf_data/'
	fname
	    'xz_realistic_z50_150_160_580nm_n1335_HR2'
	x_step
	    0.0625
	z_step
	    0.0625


## Group /trajectories

In [20]:
group = '/trajectories'

print 'Nodes in in %s:\n' % group

print S.store.data_file.get_node(group)
for node in S.store.data_file.get_node(group)._f_list_nodes():
    print '\t%s %s' % (node.name, node.shape)
    print '\t    %s' % node.title

Nodes in in /trajectories:

/trajectories (Group) 'Simulated trajectories'
	emission (15, 600000)
	    Emission trace of each particle
	emission_tot (0,)
	    Summed emission trace of all the particles
	position (15, 3, 600000)
	    3-D position trace of each particle


In [21]:
group = '/trajectories'

print 'Attributes in %s:\n' % group

print S.store.data_file.get_node(group)
for attr in S.store.data_file.get_node(group)._v_attrs._f_list():
    print '\t%s' % attr
    print '\t    %s' % type(S.store.data_file.get_node(group)._v_attrs[attr])

Attributes in /trajectories:

/trajectories (Group) 'Simulated trajectories'
	init_random_state
	    <type 'tuple'>
	last_random_state
	    <type 'tuple'>
	psf_name
	    <type 'numpy.string_'>


###`emission` attributes

In [22]:
node_name = '/trajectories/emission'
node = S.store.data_file.get_node(node_name)

print "\n%s %s: '%s'" % (node.name, node.shape, node.title)
print '\n    List of attributes:'
for attr in node.attrs._f_list():
    print '\t%s' % attr
    attr_data = repr(node._v_attrs[attr])
    if len(attr_data) > 300:
        attr_data = hash_(node._v_attrs[attr])
    print "\t    %s" % attr_data


emission (15, 600000): 'Emission trace of each particle'

    List of attributes:


###`position` attributes

In [23]:
node_name = '/trajectories/position'
if node_name in S.store.data_file:
    node = S.store.data_file.get_node(node_name)
    
    print "\n%s %s: '%s'" % (node.name, node.shape, node.title)
    print '\n    List of attributes:'
    for attr in node.attrs._f_list():
        print '\t%s' % attr
        print "\t    %s" % repr(node._v_attrs[attr])
else:
    print '%s not present.'


position (15, 3, 600000): '3-D position trace of each particle'

    List of attributes:


## Group /timestamps

In [24]:
group = '/timestamps'

print 'Nodes in in %s:\n' % group

print S.store.data_file.get_node(group)
for node in S.store.data_file.get_node(group)._f_list_nodes():
    print '\t%s' % node.name
    print '\t    %s' % node.title

Nodes in in /timestamps:

/timestamps (Group) 'Timestamps of emitted photons'
	max_rate300kcps_bg2000cps_rs_bfb8
	    Simulated photon timestamps
	max_rate300kcps_bg2000cps_rs_bfb8_par
	    Particle number for each timestamp


In [25]:
group = '/timestamps'

print 'Attributes in %s:\n' % group

print S.store.data_file.get_node(group)
for attr in S.store.data_file.get_node(group)._v_attrs._f_list():
    print '\t%s' % attr
    print '\t    %s' % type(S.store.data_file.get_node(group)._v_attrs[attr])

Attributes in /timestamps:

/timestamps (Group) 'Timestamps of emitted photons'
	last_random_state
	    <type 'tuple'>


In [26]:
group = '/timestamps'

print '>> Nodes in %s' % S.store.data_file.get_node(group)

for node in S.store.data_file.get_node(group)._f_list_nodes():
    #print '\t%s' % node.name
    #print '\t    %s' % node.title
    print "\n%s %s: '%s'" % (node.name, node.shape, node.title)
    print '\n    List of attributes:'
    for attr in node.attrs._f_list():
        print '\t%s' % attr
        attr_data = repr(node._v_attrs[attr])
        if len(attr_data) > 300:
            attr_data = hash_(node._v_attrs[attr])
        print "\t    %s" % attr_data
        
print '\n>> Attributes in %s:\n' % group

print S.store.data_file.get_node(group)
for attr in S.store.data_file.get_node(group)._v_attrs._f_list():
    print '\t%s' % attr
    print '\t    %s' % type(S.store.data_file.get_node(group)._v_attrs[attr])

>> Nodes in /timestamps (Group) 'Timestamps of emitted photons'

max_rate300kcps_bg2000cps_rs_bfb8 (889,): 'Simulated photon timestamps'

    List of attributes:
	bg_rate
	    2000.0
	clk_p
	    4.9999999999999998e-08
	init_random_state
	    bfb867eb5c5858e04685b790d6370c458b9747d6
	max_rate
	    300000.0

max_rate300kcps_bg2000cps_rs_bfb8_par (889,): 'Particle number for each timestamp'

    List of attributes:
	bg_particle
	    15
	num_particles
	    15

>> Attributes in /timestamps:

/timestamps (Group) 'Timestamps of emitted photons'
	last_random_state
	    <type 'tuple'>


In [45]:
S.store.close()

In [29]:
S.store_fname

'pybromo_7798be_D1.2e-11_15P_32pM_step0.5us_t_max0.3s_ID0-0.hdf5'

In [30]:
S = load_simulation(S.store_fname)

#Random State: reprodicibility and high-quality pseudo-random numbers

PyBromo uses Numpy's [`RandomState`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.random.RandomState.html)
object to track the current state of the random number generator at different 
simulation stages. 

Tracking the random state has a dual purpose. First, it allows to reproduce
any simulation step.

Second, it allows to mantain an high-quality pseudo-random number stream when 
the simulation is resumed. This point is more subtle. Simulation can be performed 
in different steps. When resuming a simulation to proceed to the nex step it is important 
to use the last saved random state. In fact, by resetting the random state using an arbitrary
seed we may easily introduce correlation between the previous and current stream of random numbers. 
For example streams generated with seeds 1 and 2 will be correlated 
(up to 1e6 samples!) because many bits in the seed are the same. This is a property of the 
[Mersenne twister](http://en.wikipedia.org/wiki/Mersenne_twister)
algorithm (see also [this paper](http://www.iro.umontreal.ca/~lecuyer/myftp/papers/wellrng.pdf)). 
To avoid this well-known problem we need to use a single stream by freezing (saving) and restoring 
the random state at each step.

Notice that there are 3 steps that require random numbers:

1. Generating the initial **particles position**
2. **Brownian motion** trajectories simulation (3-D trajectories + emission rates)
3. Generating **timestamps** based on the emission rates

The random state is tracked as follows:

1. When generating the particles with `gen_particles` the new `Particles` object 
will contain the `.init_random_state` attribute (and, as a convenience, a 3 digit 
hash in `.rs_hash`)
2. Whem performing the Brownian motion simulation with `.sim_brownian_motion`,
the '/trajectories' group (shortcut `S.traj_group`) will store the initial and 
the final state as group attributes: `.init_random_state` and `.last_random_state`.
The assumption is that we simulate only 1 Browian motion diffusion per object
so makes sense to store these values as group attributes.
3. When generating timestamps with `S.sim_timestamps_em_store`, we will generate
timestamps several times for different emission or background rates. Therefore
the '/timestamps' group (shortcut`S.ts_group`) contains the `last_random_state`
attribute and each timestamp array contains the `init_random_state` attribute.

> **NOTE:** Since the random-state data structure ([a tuple of a string, an array, and 3 numbers](http://docs.scipy.org/doc/numpy/reference/generated/numpy.random.get_state.html)) 
> is numpy-specific we save it without any conversion. In fact, using the same random state 
> in another programming language would require a deep understanding of the Mersenne twister 
> implementation. Not to mention that a proprietary language may not provide such level of details
> to the user. Anyway, if you are motivated to use the random state in another language, an 
> additional conversion from numpy format would be the least of your problems.

##Notebook style

In [27]:
cd $NOTEBOOK_DIR

C:\Data\Antonio\software\Dropbox\notebooks\pybromo


In [28]:
from IPython.core.display import HTML
HTML(open("./styles/custom2.css", "r").read())